In [23]:
import geopandas as gpd 
try:
    from osgeo import gdal
    print("GDAL 导入成功！")
except Exception as e:
    print("GDAL 导入失败:", e)
from osgeo import ogr
from osgeo import osr
from math import ceil
import os 
import glob
def Fishgrid(outfile,xmin,xmax,ymin,ymax,gridwidth,gridheight):
    #参数转换到浮点型
    xmin = float(xmin)
    xmax = float(xmax)
    ymin = float(ymin)
    ymax = float(ymax)
    gridwidth = float(gridwidth)
    gridheight = float(gridheight)

    #计算行数和列数
    rows = ceil((ymax-ymin)/gridheight)
    cols = ceil((xmax-xmin)/gridwidth)

    #初始化起始格网四角范围
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin+gridwidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridheight

    #创建输出文件
    outdriver = ogr.GetDriverByName('ESRI Shapefile')
    if outdriver is None:
        print("Shapefile驱动未加载！")
    if os.path.exists(outfile):
        outdriver.DeleteDataSource(outfile)
    outds = outdriver.CreateDataSource(outfile)
    outlayer = outds.CreateLayer(outfile,geom_type = ogr.wkbPolygon)
    # print(outlayer)
    #不添加属性信息，获取图层属性
    outfielddefn  = outlayer.GetLayerDefn()
    #遍历列，每一列写入格网
    col = 0
    while col<cols:
        #初始化，每一列写入完成都把上下范围初始化
        ringYtop = ringYtopOrigin
        ringYbottom = ringYbottomOrigin
        #遍历行，对这一列每一行格子创建和写入
        row = 0
        while row<rows:
            #创建左上角第一个格子
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(ringXleftOrigin,ringYtop)
            ring.AddPoint(ringXrightOrigin,ringYtop)
            ring.AddPoint(ringXrightOrigin,ringYbottom)
            ring.AddPoint(ringXleftOrigin,ringYbottom)
            ring.CloseRings()
            #写入几何多边形
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)
            #创建要素，写入多边形
            outfeat = ogr.Feature(outfielddefn)
            outfeat.SetGeometry(poly)
            #写入图层
            outlayer.CreateFeature(outfeat)
            outfeat = None
            #下一多边形，更新上下范围
            row+=1
            ringYtop = ringYtop - gridheight
            ringYbottom = ringYbottom-gridheight
        #一列写入完成后，下一列，更新左右范围
        col+=1
        ringXleftOrigin = ringXleftOrigin+gridwidth
        ringXrightOrigin = ringXrightOrigin+gridwidth
    #写入后清除缓存
    outds = None


if __name__ == "__main__":
    '''
        根据矢量图形生成格网文件
    '''

    input_dir = "E:\四个城市\四个城市\研究区"
    output_dir = r"E:\四个城市\格网\200"
    width = 200 # 格网大小
    os.makedirs(output_dir, exist_ok=True)
    shp_files = glob.glob(os.path.join(input_dir, "**", "*.shp"), recursive=True)

    for shp_file in shp_files:
        basename = os.path.basename(shp_file)
        mws_pcs=gpd.read_file(shp_file)
        mws_pcs = mws_pcs.to_crs("ESRI:102113") # web mercator投影 84

        Fishgrid('mws_pcs_gw.shp', # 临时文件
            xmin=mws_pcs.bounds.minx,
            ymin=mws_pcs.bounds.miny,
            xmax=mws_pcs.bounds.maxx,
            ymax=mws_pcs.bounds.maxy,
            gridwidth=width,
            gridheight=width)
        mws_pcs_gw=gpd.read_file('mws_pcs_gw.shp')
        mws_pcs_gw.set_crs(mws_pcs.crs,inplace=True)
        mwsgw=gpd.clip(mws_pcs_gw,mws_pcs) # 裁切
        # 转换为 WGS84
        mwsgw = mwsgw.to_crs("EPSG:4326")
        # 重新编号
        mwsgw.index=range(mwsgw.shape[0])
        mwsgw.FID=range(mwsgw.shape[0])
        # 保存
        output_file = os.path.join(output_dir, basename)
        mwsgw.to_file(output_file)

GDAL 导入成功！


C:\Users\LMQ\AppData\Local\Temp\ipykernel_202452\2812443735.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  xmin = float(xmin)
C:\Users\LMQ\AppData\Local\Temp\ipykernel_202452\2812443735.py:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  xmax = float(xmax)
C:\Users\LMQ\AppData\Local\Temp\ipykernel_202452\2812443735.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ymin = float(ymin)
C:\Users\LMQ\AppData\Local\Temp\ipykernel_202452\2812443735.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ymax = float(ymax)
C:\Users\LMQ\AppData\Local\Temp\ipykernel_202452\2812443735.py:14: FutureWar